##Check song  profile to make sure song ids match song ids in MSD
'caue the track ids don't seem to carry over  
(They do, yay! Well, except for the known mismatches, probably. Or at least I'm going to assume there are no other mismatches until I discover otherwise)  
```
http://developer.echonest.com/api/v4/song/profile?api_key=API_KEY&format=json&id=SOEBCBI12AF72A154F
```

In [5]:
import sys
import requests
import time
import pickle
from pprint import pprint

In [2]:
def pickle_stuff(filename, data):
    ''' save file '''
    with open(filename, 'w') as picklefile:
        pickle.dump(data, picklefile)

def unpickle(filename):
    ''' open file '''
    with open(filename, 'r') as picklefile:
        old_data = pickle.load(picklefile)
    return old_data

In [7]:
with open("echonestapi1.txt", 'r') as ena:
    api_key = ena.read().strip()

In [8]:
rate_time = 0.501 #3.1

In [3]:
r = requests.get('http://developer.echonest.com/api/v4/track/profile?api_key='+api_key+'&id=musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d')

In [4]:
rdata = r.json()
pprint(rdata)

{u'response': {u'status': {u'code': 0,
                           u'message': u'Success',
                           u'version': u'4.2'},
               u'track': {u'artist': u'Hammock',
                          u'catalog': u'musicbrainz',
                          u'foreign_id': u'musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d',
                          u'foreign_ids': [u'musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d'],
                          u'foreign_release_id': u'musicbrainz:release:9f2cecb3-ca9a-4d29-8bcd-a692f32e2f11',
                          u'foreign_release_ids': [u'musicbrainz:release:9f2cecb3-ca9a-4d29-8bcd-a692f32e2f11'],
                          u'id': u'TRRYMSJ13B7BF722BF',
                          u'release': u'More Dead Than Alive (Get Away From the Medicine)',
                          u'song_id': u'SOCGKJR12A58A79D30',
                          u'status': u'complete',
                          u'title': u'I Can Almost See You'}}}


In [5]:
pprint(rdata["response"]["status"]["message"])

u'Success'


In [8]:
pprint(rdata["response"]["track"])
pprint(rdata["response"]["track"]["song_id"])

{u'artist': u'Hammock',
 u'catalog': u'musicbrainz',
 u'foreign_id': u'musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d',
 u'foreign_ids': [u'musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d'],
 u'foreign_release_id': u'musicbrainz:release:9f2cecb3-ca9a-4d29-8bcd-a692f32e2f11',
 u'foreign_release_ids': [u'musicbrainz:release:9f2cecb3-ca9a-4d29-8bcd-a692f32e2f11'],
 u'id': u'TRRYMSJ13B7BF722BF',
 u'release': u'More Dead Than Alive (Get Away From the Medicine)',
 u'song_id': u'SOCGKJR12A58A79D30',
 u'status': u'complete',
 u'title': u'I Can Almost See You'}
u'SOCGKJR12A58A79D30'


In [30]:
mbid_songs = {}
skiplines = 4600

In [31]:
with open("mbid_songs.csv", 'r') as msf:
    for line in msf:
        line = line.strip()
        segs = line.split(",")
        key = segs[0]
        track = segs[1]
        mbid_songs[key] = track

In [32]:
print len(mbid_songs.keys())

916225


In [33]:
linecount = 0
with open("mbids.txt", 'r') as mf:
    for s in xrange(skiplines):
        next(mf)
    t0 = time.time()
    for line in mf:
        linecount += 1
        print len(line.split())
        t1 = time.time()
        count = 0
        for mbid in line.split():
            if mbid in mbid_songs:
                #print "continuing"
                count += 1
                continue
            print count,
            count += 1
            moveon = False
            while not moveon:
                #print "requesting"
                time.sleep(rate_time)
                try:
                    enr = requests.get('http://developer.echonest.com/api/v4/track/profile?api_key='+api_key+
                                       '&id=musicbrainz:track:'+mbid)
                    enrj = enr.json()
                    status = enrj["response"]["status"]["message"]
                    if status == "Success":
                        #print "true"
                        if "song_id" in enrj["response"]["track"]:
                            #print "success" #"ids", mbid
                            mbid_songs[mbid] = enrj["response"]["track"]["song_id"]
                        moveon = True
                    elif status == "The Identifier specified does not exist":
                        #echo nest does not have all mbids mapped to tracks
                        #print status #mbid, status
                        mbid_songs[mbid] = ""
                        moveon = True
                    else:
                        print status
                        time.sleep(3)
                except requests.exceptions.ConnectionError as e:
                    print e,
                    time.sleep(3)
        t2 = time.time()
        if t2-t1 > 0.5:
            print "\nline seconds:", t2-t1,
            print "total seconds:", t2-t0, 
            print "total lines:", linecount     


1089
656
1768
1325
929
1870
1117
1226
1582
634
652
10
836
1662
17
1204
1043
929
1683
1187
1972
995
1599
939
830
903
603
1912
902
1538
272
132
660
250
302
547
2436
1040
1658
869
1379
5
1062
341
227
680
800
929
1291
1623
1562
156
1644
1062
432
667
1962
1240
962
1281
296
1043
2249
1329
584
1805
1372
1218
602
967
241
1391
1352
484
884
1693
899
861
524
378
1630
435
1982
1619
1557
1239
244
984
1361
1485
616
694
490
775
949
453
405
1518
689
954
311
1390
1208
861
1479
1422
1761
2252
1451
1559
824
204
1436
1156
366
690
1042
2591
0
1625
506
776
542
634
1082
1010
1440
807
957
17
172
1364
2382
725
145
1845
491
1057
1700
2119
2027
1656
827
1260
1441
1038
1461
887
953
2112
657
816
273
590
790
746
2350
680
1045
541
1273
1432
307
2044
915
1033
1645
747
575
2024
1366
1149
1089
578
1469
1362
1412
0
937
788
310
611
175
1455
1364
1772
867
1568
161
532
742
1048
1286
1165
110
460
726
793
704
1131
352
1126
1124
755
1432
481
17
1147
720
1246
2100
240
361
184
673
722
676
2105
1264
686
678
599
101
1257
576
1170

KeyboardInterrupt: 

In [ ]:
print len(mbid_songs.keys())

In [29]:
with open("mbid_songs.csv", 'w') as msf:
    for k, v in mbid_songs.iteritems():
        msf.write(k+","+v+"\n")

In [116]:
#print mbid_songs

In [10]:
#pickle_stuff("mbid_song_dict.pkl", mbid_songs)